In [ ]:
import duckdb
import os

# Criar conexão em memória
con = duckdb.connect(database=':memory:')

## 🔌 Conexão Inicial

In [ ]:
# Instalação do DuckDB
!pip install duckdb -q

## 📦 Instalação e Setup

# Capítulo 01: Introdução DuckDB + S3

Este capítulo apresenta os conceitos básicos de integração entre DuckDB e Amazon S3.

In [ ]:
# 📖 Sintaxe Básica de Leitura
con.execute("""
SELECT * FROM 's3://your-bucket/filename.parquet';
""")
print(con.fetchall())

In [ ]:
# Combinar dados locais com dados em S3
con.execute("""
SELECT l.*, s.metadata
FROM local_table l
JOIN 's3://my-bucket/reference_data.parquet' s
ON l.id = s.id;
""")
print(con.fetchall())

### Join com Dados Locais

In [ ]:
# Ler de S3, transformar e escrever de volta
con.execute("""
COPY (
    SELECT user_id, sum(amount) as total
    FROM 's3://raw-data/transactions/*.parquet'
    GROUP BY user_id
) TO 's3://processed-data/user_totals.parquet';
""")
print(con.fetchall())

### ETL Simplificado

In [ ]:
# Analisar logs armazenados em S3
con.execute("""
SELECT date, count(*) as requests
FROM 's3://my-logs-bucket/2024/*/*.parquet'
GROUP BY date
ORDER BY date DESC;
""")
print(con.fetchall())

## 💼 Casos de Uso Práticos

### Análise de Logs

In [ ]:
# Ler múltiplos arquivos com padrão
con.execute("""
SELECT * FROM 's3://your-bucket/*.parquet';
""")
print(con.fetchall())

## 🔍 Leitura com Glob Patterns

In [ ]:
# Escrever dados para S3
con.execute("""
COPY table_name TO 's3://your-bucket/output.parquet';
""")
print(con.fetchall())

## ✍️ Sintaxe Básica de Escrita